## Jose Luis Padilla 

# Modulo 3 - Unidad 5
# Caso práctico: *Feature engineering* y métricas de clasificación binaria (datos de imágenes de dígitos manuscritos)

In [74]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.datasets import fetch_openml
import sklearn as sk

## Descripción del caso

En este caso práctico se muestran técnicas de feature engineering para abordar un problema de clasificación de imágenes de digitos manuscritos, así como las métricas del resultado de la clasificación. Si bien las métricas mostradas son las más habituales, las posibilidades de feature engineering son ilimitadas, así que lo que se muestra no es más que un ejemplo para ilustrar los conceptos. 
Los datos a analizar corresponden al conjunto de datos MNIST, de imágenes normalizadas de dígitos manuscritos. El problema de clasificación de estos dígitos fue durante mucho tiempo un problema de referencia para comparar algoritmos de clasificación, pero con las técnicas actuales es relativamente sencillo conseguir una precisión del 100% y por lo tanto ahora se usan problemas de referencia más complejos. 

## Acceso a los datos y preparación

La libraría scikit_learn proporciona funciones de utilidad para acceder a conjuntos de datos de referencia (en este caso, a través del sitio de datos OpenML), que es una manera sencilla de disponer de datos interesantes para problemas de este tipo.

El campo "target" contiene un dataframe con el dígito que corresponde a cada caso, y el campo "data" contiene los pixels de la imagen del dígito. Copiamos los datos en nuevas variables para no tener que volver a descargar los datos si las modificamos por error, y vemos el tamaño de los datos y la estructura del dataframe mostrando sus primeras filas

(En este ejemplo, este medio de acceso a los datos está comentado, y se sustituye por un acceso local, para que sea posible ejecutar el notebook aún sin conexión a internet o aunque el sitio openml.org deje de estar disponible. De todos modos se muestra aquí con fines didácticos)

In [75]:
# # Descargamos el conjunto de datos mnist_784 y pedimos que nos devuelva los datos como dataframes de Pandas

# bunch = fetch_openml(name = 'mnist_784', as_frame = True)

# clases = bunch.target.copy()
# digitos_raw = bunch.data.copy()

In [76]:
# Descargamos el conjunto de datos desde ficheros locales
clases = pd.read_parquet('D:\\papi\\BIG DATA\\MASTER\\Modulo 03\\Ejercicios\\Digitos\\mnist_clases.parquet').iloc[:,0]
digitos_raw = pd.read_parquet('D:\\papi\\BIG DATA\MASTER\\Modulo 03\\Ejercicios\\Digitos\\mnist_pixels.parquet')

In [77]:
clases.shape

(70000,)

In [78]:
digitos_raw.shape

(70000, 784)

In [79]:
digitos_raw.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Tenemos 70.000 casos, para cada uno de los cuales tenemos los 784 pixels de la imagen (28 x 28). Los valores que se ven las filas mostradas son todos cero porque corresponden a las esquinas de algunas de las imágenes, pero entre medias hay muchos datos con otros valores, como se verá más adelante. Para que sea más sencillo trabajar con ellos, vamos a transformar cada serie de datos de pixels en valores asociados a una coordenada "x" y una coordenada "y" (los primeros 28 pixels corresponde a    "y=0"    y    "x de 0 a 27", los siguientes a    "y=1"    y    "x de 0 a 27",    etc.)

In [80]:
# Creamos un índice multinivel con las coordenadas de cada pixel, y se lo asignamos al dataframe
indice = pd.MultiIndex.from_tuples([(i,j) for j in range(28) for i in range(28)], names=['x', 'y'])
digitos_wide = digitos_raw
digitos_wide.columns = indice

# Le asignamos el nombre "Caso" al índice de filas, para luego poder hacer referencia al dataset "clases"
digitos_wide.index.name = 'Caso'

digitos_wide.head()

x,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
y,0,0,0,0,0,0,0,0,0,0,...,27,27,27,27,27,27,27,27,27,27
Caso,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Estos datos están en lo que se conoce como "__wide format__", formato ancho. Este formato puede ser útil para mostrar los datos y para hacer operaciones matriciales, pero para análisis exploratorio  y para visualización suele ser más conventiente poner los datos en "__long format__", esto eso, poniendo cada variable como una columna distinta. Pandas nos ofrece una forma relativamente sencilla de transformar los datos de un formato al otro, según sea necesario

In [81]:
# "unstack" transforma los niveles del índice de columnas en niveles del índice de filas; a continuación
# reordenamos los niveles para dejar "Caso" como primer nivel, damos un nombre a la nueva variable,
# ordenamos según el nuevo índice de filas y, con "reset_index()", transformamos los niveles del índice
# de filas en nuevas columnas.
digitos_long = digitos_wide.unstack().reorder_levels([2, 0, 1]).rename('Valor').sort_index().reset_index()

digitos_long.head()

,Caso,x,y,Valor
0,0,0,0,0.0
1,0,0,1,0.0
2,0,0,2,0.0
3,0,0,3,0.0
4,0,0,4,0.0


Para facilitar el diagnóstico de lo que hagamos a continuación, creamos una función auxiliar para visualizar los dígitos. Esto lo hacemos simplemente haciendo un gráfico en el que se muestra el valor de cada pixel en su posición x, y.

In [116]:
def mostrar_casos(casos, titulo = ''):
    
    data = digitos_long[digitos_long.Caso.isin(casos)]
    
    return alt.Chart(data, width=100, height=100, title = titulo)\
            .mark_rect()\
            .encode(x = alt.X('x:O', axis = None),
                    y = alt.Y('y:O', axis = None),
                    color = alt.Color('Valor:Q', scale = alt.Scale(scheme="blues"), legend = None),
                    facet = alt.Facet('Caso:O', columns = 6, title = None))

# Desactivamos el aviso que da altair en caso de que la cantidad de datos a representar sea muy alta (para aquellos casos en los
# que se desea hacer gráficos, generalmente interactivos, en los que se muestran decenas de miles o incluso millones de datos,
# altair ofrece la posibilidad de generarlos dinámicamente en vez de incluirlos en el gráfico, pero esa posibilidad ahora no nos interesa
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [118]:
mostrar_casos(range(0, 22))

alt.Chart(...)

In [119]:
# Mostramos el contenido del dataframe "clases" para los mismos casos; el primer valor en la tabla de resultados
# es el número del caso, y el segundo el dígito al que corresponde
clases.loc[range(0, 22)]

0     5
1     0
2     4
3     1
4     9
5     2
6     1
7     3
8     1
9     4
10    3
11    5
12    3
13    6
14    1
15    7
16    2
17    8
18    6
19    9
20    4
21    0
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

Como se ve, mediante la funcion mostrar_casos vemos la representación de las imágenes contenidas en el dataframe "digitos_long", y en el dataframe "clases" tenemos la clase a la que pertenece cada imagen, esto es, el dígito concreto que está representado en esa imagen. Algunos dígitos manuscritos son muy fáciles de identificar, pero otros pueden dar lugar a confusión (como el caso 11 mostrado más arriba, que según conjunto de datos es un 5, pero tal vez podría confundirse con un 6)

# Feature engineering para clasificación de dígitos

Vamos a tratar de definir alguna feature interesante. Para ello, vamos a limitar el problema a clasificar los dígitos 0 y 1

In [120]:
# Seleccionamos los registros del dataframe "clases" que corresponden a dígitos 0 y 1,
clases01 = clases[clases.isin(['0','1'])].rename('Valor real')
casos1 = clases01[clases01 == '1'].index.tolist()
casos0 = clases01[clases01 == '0'].index.tolist()

# y filtramos el dataframe "digitos_long" por los casos que quedaron en "clases01"
digitos01 = digitos_long[digitos_long.Caso.isin(casos0 + casos1)]

¿Qué característica de las imágenes, fácil de calcular, nos podría resultar útil para distinguir ceros de unos? Esta es el tipo de preguntas al que se trata de responder con el feature engineering. Partimos de la idea de que los unos son "más estrechos" que los ceros, y tratamos de representar eso en un valor. Para ello, calculamos el valor máximo de los pixels de cada línea vertical de cada imagen y calculamos la media de esos valores para cada caso. En el caso de los unos, habrá relativamente pocas líneas verticales con valores distintos de cero, y por lo tanto la media será más pequeña.

In [121]:
# Calculamos el máximo del valor de los pixel para cada línea vertical de cada caso 
sumas_verticales = digitos01.groupby(['Caso', 'x'])['Valor'].max()

# Calculamos la media de esos valores para cada caso
media_sumas_verticales = sumas_verticales.groupby(['Caso']).mean()

Vamos a comprobar si esta "feature" que hemos creado tiene alguna relación con si una imagen es un cero o un uno. Una manera sencilla es comprobar la media del indicador para cada tipo de casos, o, para verlo con más detalle, hacer un histograma los valores del indicador para cada tipo de caso

In [122]:
# Calculamos la media de ese valor para las imágenes de ceros y las imágenes de unos
media_sumas_verticales.loc[casos0].mean(), media_sumas_verticales.loc[casos1].mean()

(155.78295668549856, 73.16317397849069)

In [130]:
# Hacemos un histograma de valores del indicador para las imágenes de ceros y las imágenes de unos
data = pd.concat([clases01, media_sumas_verticales.rename('media_sumas_verticales')], axis = 1)

alt.Chart(data, width=500, height=120,).mark_bar().encode(
    x=alt.X('media_sumas_verticales:Q', bin=alt.Bin(maxbins = 40)),
    y=alt.Y('count()', title = None),
    row = 'Valor real:N'
)

alt.Chart(...)

Otra páctica interesante es ver algunos ejemplos de casos y el valor correspondiente de la feature, o examinar los casos en los que la feature tiene valores extremos, para hacerse una idea de "qué representa". En este caso, se puede observar que la feature que hemos generado recoge la idea que teníamos: valores más altos corresponden a figuras más anchas, y valores más bajos a figuras más estrechas.

In [131]:
# Mostramos algunos casos concretos...
mostrar_casos(clases01.index.tolist()[15:21])

alt.Chart(...)

In [132]:
# ... y comprobamos el valor de esa feature para cada una de esas imágenes
media_sumas_verticales.loc[clases01.index.tolist()[15:21]].to_dict()

{67: 76.46428571428571,
 68: 182.14285714285714,
 69: 129.82142857142858,
 70: 103.39285714285714,
 72: 32.785714285714285,
 75: 134.17857142857142}

In [133]:
# Mostremos los casos extremos...
mostrar_casos(media_sumas_verticales.nlargest(6).index.tolist(), 'Casos con los valores más altos de media_sumas_verticales')

alt.Chart(...)

In [134]:
mostrar_casos(media_sumas_verticales.nsmallest(6).index.tolist(), 'Casos con los valores más bajos de media_sumas_verticales')

alt.Chart(...)

Tanto por la media de los valores como comprobando algunos casos individuales, vemos que, tal como esperábamos, el valor de esa feature es más alto para los ceros que para los unos, así que tal vez pueda utilizarse para clasificar los casos.

# Métricas de clasificación binaria

Para clasificar las imágenes, vamos a utilizar un modelo extremadamente sencillo: nuestra predicción será un uno si el valor del media_sumas_verticales es menor de un cierto umbral, y un cero si es mayor.

In [135]:
# Creamos una nueva serie de datos a partir de std_sumas_verticales, con un '1' o un '0' en función de si el valor es mayor o menor que el umbral

umbral = 120
clases01_pred = media_sumas_verticales.apply(lambda x: '1' if x < umbral else '0').rename('Predicción')

Hacemos una tabla de contingencia comparando nuestras predicciones con los valores reales, y calculamos algunas métricas de nuestro clasificador binario trivial (considerando como "positivo" de nuestro clasificador los ceros)

In [136]:
# La funcion crosstab de Pandas hace un conteo cruzado de casos entre dos series de datos. Como en este caso los valores posibles son dos, ese conteo no es otra cosa que una tabla de contingencia

tabla_contingencia = pd.crosstab(clases01_pred, clases01)
tabla_contingencia

Valor real,0,1
Predicción,,
0,6290,889
1,613,6988


In [137]:
verdaderos_positivos = tabla_contingencia['0']['0']
verdaderos_negativos = tabla_contingencia['1']['1']

falsos_positivos = tabla_contingencia['1']['0']
falsos_negativos = tabla_contingencia['0']['1']

In [138]:
sensibilidad = verdaderos_positivos / (verdaderos_positivos + falsos_negativos)
especificidad = verdaderos_negativos/ (verdaderos_negativos + falsos_positivos)
precision = verdaderos_positivos / (verdaderos_positivos + falsos_positivos)

In [139]:
sensibilidad, especificidad, precision

(0.9111980298420976, 0.8871397740256443, 0.8761665970190834)

Vemos como, simplemente creando una feature sencilla, hemos conseguido un clasificador binario con una sensibilidad y una especificidad bastante altas (o, en otras palabras, capaz de detectar la mayoría de los ceros y capaz de equivocarse pocas veces cuando dice que no es un cero)

El cálculo de las métricas de un clasificador binario, como el que acabamos de hacer, es una necesidad muy común, y naturalmente lo habitual no es hacerlo manualmente, sino utilizar alguna librería estándar (en este caso, scikit-learn) Conviene recordar la equivalencia de términos para interpretar los resultados que se muestran: "sensibilidad" se corresponde el "recall" de la clase positiva, y especificidad con el "recall" de la clase negativa.

In [140]:
# scikit-learn tiene una función específica para generar las métricas de clasificación a partir de los
# casos reales y las predicciones

from sklearn.metrics import classification_report

print(classification_report(clases01, clases01_pred, digits = 3))

              precision    recall  f1-score   support

           0      0.876     0.911     0.893      6903
           1      0.919     0.887     0.903      7877

    accuracy                          0.898     14780
   macro avg      0.898     0.899     0.898     14780
weighted avg      0.899     0.898     0.898     14780



¿Qué pasaría si cambiásemos el valor del umbral?

In [141]:
umbral2 = 100
clases01_pred2 = media_sumas_verticales.apply(lambda x: '1' if x < umbral2 else '0').rename('Predicción')

pd.crosstab(clases01_pred2, clases01)

Valor real,0,1
Predicción,,
0,6783,1768
1,120,6109


In [142]:
print(classification_report(clases01, clases01_pred2))

              precision    recall  f1-score   support

           0       0.79      0.98      0.88      6903
           1       0.98      0.78      0.87      7877

    accuracy                           0.87     14780
   macro avg       0.89      0.88      0.87     14780
weighted avg       0.89      0.87      0.87     14780



Vemos como usando un umbral de clasificación distinto han cambiado las métricas. El resultado debería ser sencillo de interpretar (especialmente si imaginamos nuestro clasificador como un "detector de ceros", ya que ese es el sentido de asignar el cero como "caso positivo"): cuando hemos disminuido el umbral, aceptamos como "ceros" a más valores, lo que aumenta la sensibilidad ("se nos pasan" menos casos positivos sin detectar) pero disminuye la especificidad (damos como positivos más casos que no lo son)

## Curva ROC y métrica AUC ROC

Como vemos, cambiando el umbral de detección es posible aumentar la sensibilidad sacrificando especificidad, o viceversa. Según la aplicación que se quiera dar al clasificador, será más importante una métrica o otra, y se utilizará un umbral distinto, aunque sea el mismo clasificador. Vamos a tratar de calcular todas las combinaciones posibles de sensibilidad y especificidad

In [143]:
# Creamos unas funciones sencillas que nos permiten calcular los parámetros de la curva ROC de manera similar a como se ha hecho en casos anteriores
def calcular_prediccion(umbral, valores):
    return valores.apply(lambda x: '1' if x < umbral else '0').rename('Predicción')

def calcular_TPR_FPR(umbral, valores = media_sumas_verticales):
    pred = calcular_prediccion(umbral, valores)
    tabla_vacia = pd.DataFrame(0., index = ['0', '1'], columns= ['0', '1'])
    tabla_contingencia = tabla_vacia + pd.crosstab(pred, clases01)
    
    TP = tabla_contingencia['0']['0']
    TN = tabla_contingencia['1']['1']

    FP = tabla_contingencia['1']['0']
    FN = tabla_contingencia['0']['1']
    
    return (TP/(TP + FN), FP/(FP + TN))   

In [144]:
# Calculamos los valores de TPR (true positive rate) y FPR (false positive rate) para valores del umbral en el rango entre 200 y 1400, 
# y lo almacenamos en un dataframe con los nombres de columna correspondientes
roc = pd.DataFrame([calcular_TPR_FPR(umbral) for umbral in np.linspace(media_sumas_verticales.min(), media_sumas_verticales.max(), 200)],
                   columns = ['TPR (Sensibilidad)', 'FPR (1-Especificidad)'])

In [103]:
# Generamos un sencillo gráfico con esos valores, lo que constituye la curva ROC de nuestro clasificador
alt.Chart(roc, width=250, height=250).mark_line()\
            .encode( x = 'FPR (1-Especificidad):Q', y = 'TPR (Sensibilidad):Q',)

alt.Chart(...)

Vemos la curva ROC de nuestro clasificador. Una curva ROC "muy pegada" a la esquina superior izquierda es propia de un clasificador muy eficaz: es posible conseguir detectar un porcentaje muy alto de los casos positivos (sensibilidad alta) manteniendo un porcentaje pequeño de falsos positivos (especificidad alta, o FPR bajo).

In [149]:
# Calculamos las diferencias de FPR con el valor anterior y con el valor posterior, para integrar por rectángulos
roc['Dif_post_FPR'] = - roc.iloc[:, 1].diff(1)
roc['Dif_prev_FPR'] = roc.iloc[:, 1].diff(-1)

# Calculamos el área bajo la curva sumando el área de los rectángulos que van entre dos valores consecutivos de FPR y entre cero y el valor de TPR,
# tanto usando el valor de TPR del lado izquierdo como usando el del lado derecho , para luego hacer la media
roc_auc_post = (roc['TPR (Sensibilidad)'] * roc['Dif_post_FPR']).sum()
roc_auc_prev = (roc['TPR (Sensibilidad)'] * roc['Dif_prev_FPR']).sum()

roc_auc = (roc_auc_post + roc_auc_prev) / 2
roc_auc

0.9626461043233324

Ese valor de AUC_ROC, próximo a uno, corresponde a un clasificador bastante efectivo. Obviamente, dado que esta es una métrica muy usada, también hay formas estándar de calcularla.

In [150]:
from sklearn.metrics import roc_auc_score
# Calculamos el área bajo la curva ROC, proporcionandole una lista de si cada caso es positivo o no y 
# un valor que debe ser mayor según crezca la probabilidad de que el caso sea positivo
roc_auc_score(clases01 == '0' , media_sumas_verticales)

0.9627845688668553

# Comparación de modelos

Vamos a generar otros modelos de clasificación para el mismo problema, para ver cómo se comportan. Un primer caso, trivial, sería asignar una probabilidad al azar a cada caso posible.

In [151]:
valores_azar = pd.Series(np.random.random_sample(clases01.shape))

In [152]:
roc2 = pd.DataFrame([calcular_TPR_FPR(umbral, valores_azar) for umbral in np.linspace(valores_azar.min(), valores_azar.max(), 200)],
                   columns = ['TPR (Sensibilidad)', 'FPR (1-Especificidad)'])

alt.Chart(roc2, width=250, height=250, title = 'ROC de un clasificador aleatorio').mark_line()\
            .encode( x = 'FPR (1-Especificidad):Q', y = 'TPR (Sensibilidad):Q',)

alt.Chart(...)

In [153]:
roc_auc_score(clases01 == '0' , valores_azar)

0.4923960823049136

Como se ve, la curva ROC de un clasificador aleatorio es (aproximadamente) una línear recta entre (0,0) y (1,1), o, en otras palabras, que se usen como se usen los valores aleatorios, siempre habrá la misma tasa de verdaderos positivos que de falsos positivos.

Veamos otro clasificador más interesante, basado en calcular la suma de los valores de todos los puntos de la imagen, que sería algo así como calcular "la cantidad de tinta" que hay en la imagen

In [154]:
cantidad_tinta = digitos01.groupby(['Caso'])['Valor'].sum()

In [155]:
roc3 = pd.DataFrame([calcular_TPR_FPR(umbral, cantidad_tinta) for umbral in np.linspace(cantidad_tinta.min(), cantidad_tinta.max(), 200)],
                   columns = ['TPR (Sensibilidad)', 'FPR (1-Especificidad)'])

alt.Chart(roc3, width=250, height=250, title = 'ROC del clasificador \"Cantidad de tinta\"').mark_line()\
            .encode( x = 'FPR (1-Especificidad):Q', y = 'TPR (Sensibilidad):Q',)

alt.Chart(...)

In [156]:
roc_auc_score(clases01 == '0' , cantidad_tinta)

0.9832475281669782

El clasificador basado en "la cantidad de tinta" muestra una curva ROC más pegada a la esquina superior izquierda, y un ROC AUC más alto, que el clasificador basado en la media de las sumas verticales, y por lo tanto podemos considerar que es un clasificador más preciso, en caso de que tuviéramos que elegir entre los dos (por supuestos, cada uno de estos clasificadores solo tiene en cuenta una única 'feature', y sería posible hacer clasificadores más sofisticados combinando estas 'features' y muchas otras)

Finalmente, vamos a comparar los errores de tipo I (falsos positivos) y de tipo II (falsos negativos) para cada uno de los clasificadores. Para ello, seleccionamos los casos negativos con un valor del 'feature' de predicción más alto, que por lo tanto es más probable que se clasificaran como positivos y dieran lugar a errores de tipo I, y los casos positivos con un valor más bajo, que darían errores de tipo II

In [157]:
mostrar_casos(media_sumas_verticales.loc[casos1].nlargest(6).index.tolist(),
              'Errores de tipo I para el clasificador \"Media de sumas verticales\"')

alt.Chart(...)

In [158]:
mostrar_casos(media_sumas_verticales.loc[casos0].nsmallest(6).index.tolist(),
              'Errores de tipo II para el clasificador \"Media de sumas verticales\"')

alt.Chart(...)

In [159]:
mostrar_casos(cantidad_tinta.loc[casos1].nlargest(6).index.tolist(),
              'Errores tipo I para el clasificador \"Cantidad de tinta\"')

alt.Chart(...)

In [160]:
mostrar_casos(cantidad_tinta.loc[casos0].nsmallest(6).index.tolist(),
              'Errores tipo II para el clasificador \"Cantidad de tinta\"')

alt.Chart(...)

Se puede distinguir claramente que los casos en que se equivoca cada clasificador son distintos (y por lo tanto, combinando esas features sería posible generar un clasificador mejor que cualquier de ellos). Entendiendo cómo funciona cada uno de los clasificadores debería ser sencillo interpretar por qué se equivoca cada uno de ellos en los casos que se equivoca, y pensar en posibles nuevas features o formas de combinarlas para seguir mejorando el modelo de clasificación, para que luego se pudiera utilizar en nuevos casos para hacer predicciones.